In [40]:
from pyplasm import *
from numpy import *
def telaio2D(px, py, bx, bz, dpDato, dbDato, beamPriority):

	dp = [py]
	for val in dpDato:
		dp.append(-val+bz)
		dp.append(py)
	db = []
	for val in dbDato:
		db.append(-val+py)
		db.append(bz)

	dpn = []
	dbn = []

	dbSum = 0
	for val in db:
		dbSum += abs(val)
		dbn.append(-val)
		
	dpSum = 0
	for val in dp:
		dpSum += abs(val)
		dpn.append(-val)
		
	x = QUOTE(dp)
	if beamPriority:
		xa = QUOTE(dbn)
	else: 
		xa = QUOTE([dbSum])
	a = PROD([x, xa])
	
	ya = QUOTE(db)
	if beamPriority:
		y = QUOTE([dpSum])
	else: 
		y = QUOTE(dpn)
	
	b = PROD([y, ya])
	
	ab = STRUCT([a,b])
	za = QUOTE([px])
	zb = QUOTE([bx])
	#abc = PROD([z, ab])
	abc = STRUCT([T(1)(-px/2), PROD([za, a]), T(1)(-(bx-px)/2),PROD([zb,b])])
	return abc

def equalsVec2D(a, b):
    if a[0] == b[0]:
        if a[1] == b[1]: return 1
    else: return 0
    
def ggpl_bone_structure(file_name):
    import csv
    dim = []
    disPillar = []
    disBeam = []
    trasl = [0, 0, 0]
    struct = STRUCT([CUBE(0)])
    vecX = []
    matchMatrix = []
    tmp = []
    tmpRow = []
    with open (file_name, 'rb') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
        j = 0
        for row in spamreader:
                j += 1
                if j % 2 == 0:
                    tmp = []
                    i = 1
                    for val in row:
                        if val == ";": i += 1
                        else: 
                            if i == 1: dim.append(float(val))
                            if i == 2: disPillar.append(float(val))
                            if i == 3: disBeam.append(float(val))
                            if i == 4: bPriority = (int(val))
                    struct = STRUCT([struct, T([1,2,3])(trasl), telaio2D(dim[0], dim[1], dim[2], dim[3], disPillar, disBeam, 1)])
                    sumZ = trasl[2]
                    for z in disBeam:
                        sumZ += z
                        sumY = trasl[1]
                        tmp.append([sumY, sumZ])
                        for y in disPillar:
                            sumY += y
                            tmp.append([sumY, sumZ])
                    if len(vecX) == 1: vecX[len(vecX)-1] -= dim[0]/2
                    else: vecX [len(vecX)-1] -= dim[0]
                    vecX.append(-dim[0])
                    for val in matchMatrix:
                        for v in tmp:
                            if equalsVec2D(val, v) == 1:
                                c = CUBOID([tmpRow[0], dim[1], dim[3]])
                                struct = STRUCT([struct, T([1,2,3])([trasl[0]-tmpRow[0]+(dim[0]/2), val[0], val[1]-dim[3]]), c])
                    matchMatrix = tmp
                    dim = []
                    disPillar = []
                    disBeam = []
                else: 
                    z = 0
                    vecX.append(float(row[z]))
                    tmpRow = []
                    for val in row:
                        trasl[z] += float(val)
                        tmpRow.append(float(val))
                        z += 1
#VIEW(telaio2D(.3, .3, .4, .4, [1, 2, 1, 2], [1, 3, 4], FALSE))
    coverx = QUOTE([5.6])
    covery = QUOTE([.4])
    coverz = QUOTE([21])
    coverXZ = PROD([PROD([coverx, covery]), coverz])
    struct = STRUCT([struct, T([1,2])([9.2, 12.4]), coverXZ])
    coverx = QUOTE([.4])
    covery = QUOTE([1.9])
    coverz = QUOTE([21])
    coverYZ = PROD([PROD([coverx, covery]), coverz])
    coverYZ = STRUCT([coverYZ, T(2)(5.7), coverYZ])
    coverx = QUOTE([.4])
    covery = QUOTE([4.6])
    coverz = QUOTE([3.4])
    coverYZ = STRUCT([coverYZ, T([2, 3])([1.5, 11.6]), PROD([PROD([coverx, covery]), coverz])])
    struct = STRUCT([struct, T([1,2])([9.2, 12.4]), coverYZ])
    return struct

VIEW(ggpl_bone_structure("frame_data_440252.csv"))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000A8B8BD0> >